# Lab | APIS

In this lab, you will collect historical data about the Nobel Prize winners using [this free and non-authenticated API](https://www.nobelprize.org/organization/developer-zone-2/). According to the documentation available [here](https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1#/default/get_nobelPrizes). The base url is: "http://api.nobelprize.org/2.1/" followed by a string to specify what kind of information do you want to retrieve. The acceptable options are:

* nobelPrices
* nobelPrice/category/year
* laureates
* laureate/laureateID

# Getting the information using requests

Use the Python `requests`, and `json` libraries to obtain the information of ALL the Nobel Prizes. Make sure to verify that you get the proper status code (200).

The json outputs are simple plain text that need to be converted into the corresponding nested dictionary. Use the `.json()` method to cast the output into a Python dictionary.

Use the Pandas library to collect all the information into a Panda's DataFrame.

In [46]:
import requests
import json
import pandas as pd

url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
        print("All good!")
        print("==============")
        print("\n")

info = response.json()
response_df = pd.json_normalize(info)

response_df

All good!




nobelPrizes  meta.offset  meta.limit  \
0  [{'awardYear': '1901', 'category': {'en': 'Che...            0      100000   

   meta.count                                         meta.terms  \
0         670  https://www.nobelprize.org/about/terms-of-use-...   

                                        meta.license  \
0  https://www.nobelprize.org/about/terms-of-use-...   

                                     meta.disclaimer  \
0  https://www.nobelprize.org/about/terms-of-use-...   

                                         links.first  \
0  http://nobel-external-api-app.azurewebsites.ne...   

                                          links.self  \
0  http://nobel-external-api-app.azurewebsites.ne...   

                                          links.last  
0  http://nobel-external-api-app.azurewebsites.ne...

# Processing the output

Process the Pandas DataFrame in order to have only the following columns:

- category
- dateAwarded (as DateTime in "yyyy-mm-dd" format)
- prizeAmount
- prizeAmount
- Number_of_laureates
- motivation
- laureate_ids (as a list)

In [119]:
url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
    prizes_list = response.json()['nobelPrizes']

nobel_prizes_df=pd.json_normalize(prizes_list)

laureates_df = pd.json_normalize(nobel_prizes_df['laureates'])
nobel_prizes_df['laureate_ids'] = nobel_prizes_df['laureates'].apply(lambda x: [laureate['id'] for laureate in x] if isinstance(x, list) else [])


nobel_prizes_df[['category.en','dateAwarded','prizeAmount','laureates','topMotivation.en','laureate_ids']]


category.en dateAwarded  prizeAmount  \
0                 Chemistry  1901-11-12       150782   
1                Literature  1901-11-14       150782   
2                     Peace  1901-12-10       150782   
3                   Physics  1901-11-12       150782   
4    Physiology or Medicine  1901-10-30       150782   
..                      ...         ...          ...   
665       Economic Sciences  2023-10-09     11000000   
666              Literature  2023-10-05     11000000   
667                   Peace  2023-10-06     11000000   
668                 Physics  2023-10-03     11000000   
669  Physiology or Medicine  2023-10-02     11000000   

                                             laureates topMotivation.en  \
0    [{'id': '160', 'knownName': {'en': 'Jacobus H....              NaN   
1    [{'id': '569', 'knownName': {'en': 'Sully Prud...              NaN   
2    [{'id': '462', 'knownName': {'en': 'Henry Duna...              NaN   
3    [{'id': '1', 'knownName': {'en': 'Wilhelm Conr...              NaN   
4    [{'id': '293', 'knownName': {'en': 'Emil von B...              NaN   
..                                                 ...              ...   
665  [{'id': '1034', 'knownName': {'en': 'Claudia G...              NaN   
666  [{'id': '1032', 'knownName': {'en': 'Jon Fosse...              NaN   
667  [{'id': '1033', 'knownName': {'en': 'Narges Mo...              NaN   
668  [{'id': '1026', 'knownName': {'en': 'Pierre Ag...              NaN   
669  [{'id': '1024', 'knownName': {'en': 'Katalin K...              NaN   

           laureate_ids  
0                 [160]  
1                 [569]  
2            [462, 463]  
3                   [1]  
4                 [293]  
..                  ...  
665              [1034]  
666              [1032]  
667              [1033]  
668  [1026, 1027, 1028]  
669        [1024, 1025]  

[670 rows x 6 columns]

# Getting a Pandas DataFrame with the details of awarded authors/institutions

If you dive deeper and use the API to retrieve the details of some laureate_ids, you will notice that not allways the Nobel Prize was awarded to individuals. In some cases, the awards were given to institutions.

Get the unique ids from the previous datasets and prepare the following functions:

- get_name(laureate) ( it should return the english name 'fullName' of the individual or 'orgName' of the institution )

- get_gender(laureate) ( it should return the gender or 'Unknown' for individuals, and 'None' for institutions )

- get_birthdate(laureate) ( it should return the birthdate when it's avaialble or 'Unknown' otherwise )

- get_age(laureate) ( it should return the age of the awarded individual or 'Unknown' when it's not avaialble or for institutions )

- get_city(laureate) ( it should return the english name of the city when it's available or 'Unknown' otherwise )

- get_country(laureate) ( it should return the english name of the country when it's available or 'Unknown' otherwise )

- get_continent(laureate) ( it should return the english name of the continent when it's available or 'Unknown' otherwise )

- get_latitude(laureate) ( it should return the city's latitude when it's available or 'Unknown' otherwise )

- get_longitude(laureate) ( it should return the city's longitude
 when it's available or 'Unknown' otherwise )

Create the following dictionaries:

```python
laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}                        

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}
```

For each unique `laureate_id` of the previous DataFrame make an API call to get the details of the awarded individual/intitution and iterate of the previous dictionaries keys in order to add the corresponding information of each `laureate_id` in the empty lists of `laureates_dict`.

Finally, create a Pandas DataFrame named `laureates_df` using the `laureates_dict`.

In [80]:
import time
from tqdm import tqdm

url_laureates = "http://api.nobelprize.org/2.1/laureate/laureateID?limit=100000"

response_laureates = requests.get(url_laureates)

response_laureates.status_code

laureate_response = response_laureates.json()
laureate_response_df = pd.json_normalize(laureate_response)

laureate_response_df

ids = [int(item) for l in df['laureate_ids'].values for item in l]
unique_ids = set(ids)

def get_name(laureate):
    
   for i in range(len(laureates_df)):
        laureate_info = laureates_df.iloc[i, 0]
        if laureate_info['id'] == laureate:
            laureate_name = laureate_info['knownName.en']
        else:
            print("The ID you entered does not exist")
    return laureate_name


def get_gender(laureate):
    filtered_laureate_df=laureate_response_df[laureate_response_df['id']=='laureate']
    return filtered_laureate_df['gender']


def get_birthdate(laureate):

    birth_date = "Unknown"

    # Person
    # Your code here

    # Organization
    # Yuor code here

    return birth_date

def get_age(laureate):

    birth_date = "Unknown"
    award_date = "Unknown"

    # Person
    # Your code here

    # Organization
    # Yuor code here

    # Award date
    # Your code here

def get_city(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here

def get_country(laureate):

    # Person
    # Your code here

    # Origanization
    # Your code here


def get_continent(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here

def get_latitude(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here

def get_longitude(laureate):

    # Person
    # Your code here

    # Organization
    # Yuor code here


laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}

for index, id in enumerate(tqdm(unique_ids)):

    url = "https://api.nobelprize.org/2/laureate/" + str(id)
    response = requests.get(url)

    if response.status_code == 200:

        laureate = response.json()

        # Your code here

laureates_df = pd.DataFrame(laureates_dict)

laureates_df



IndentationError: expected an indented block after function definition on line 17 (3907341085.py, line 25)

# Country ranking

Get a ranking countries by the number of times that they had been awarded in any category.

In [ ]:
# Your code here

ID
Country                     
USA                      296
United Kingdom            91
Germany                   84
France                    63
Russia                    30
...                      ...
Greece                     1
Ghana                      1
Faroe Islands (Denmark)    1
Ethiopia                   1
Latvia                     1

[86 rows x 1 columns]